In [1]:
import sys
import xarray as xr
import numpy as np
import datetime 
import time
from datetime import datetime 
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pn
import seaborn as sns
#import sklearn
from sklearn.metrics import median_absolute_error, mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression, RANSACRegressor, HuberRegressor


import pyOptimalEstimation as pyOE

import cartopy.crs as ccrs
from cmocean import cm as cmo

import matplotlib.ticker as mticker

from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [2]:
sys.path.append('lib')
#import nonScatMWRadTran
import supporting_routines
import supporting_routines_m 
sys.path.append('/usr/people/echeverr/Documents/code/nwpsaf/rttov12/wrapper')
import pyrttov
import os

rttov_installdir = '/usr/people/echeverr/Documents/code/nwpsaf/rttov12'


/nobackup/users/echeverr/anaconda3/envs/pyOpEst/lib/python3.7/typing.py:847: FutureWarning: xarray subclass customProfile should explicitly define __slots__
  super().__init_subclass__(*args, **kwargs)


In [3]:
prior_all = supporting_routines_m.concatDiverse()

In [4]:
# Number of levels for the profiles in the priors

nlevels = len(prior_all['height'])
nprofiles = 1

In [5]:
# Channels corrispondence:

# from: D. B. Kunkee et al, "Design and Evaluation of the First 
# Special Sensor Microwave Imager/Sounder", IEEE Trans. Geosc. Rem. Sens. 
# Vol. 46, NO. 4, April 2008.
# Table 1
# Ch. 1 - 50.3 GHz Hpol (check note on pol at source for this channel) 
# Ch. 12 - 19.35 GHz Hpol
# Ch. 13 - 19.35 GHz Vpol
# Ch. 14 - 22.235 GHz Vpol
# Ch. 15 - 37 GHz Hpol
# Ch. 16 - 37 GHz Vpol

ssmiRttov = pyrttov.Rttov()

# SSMIS:
nchan_ssmi = 14
chan_list_ssmi = (1,2,3,4,5,6,7,12,13,14,15,16,23,24) #(1,12,13,14,15,16) #  #
ssmiRttov.FileCoef = '{}/{}'.format(rttov_installdir,
                                    "rtcoef_rttov12/rttov7pred54L/rtcoef_dmsp_18_ssmis.dat")

# WindSat:
#nchan_ssmi = 16
#chan_list_ssmi = (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16) #(1,2,3,4,7,8,11,12,13,14) #(1,12,13,14,15,16) #  #
#ssmiRttov.FileCoef = '{}/{}'.format(rttov_installdir,
#                                    "rtcoef_rttov12/rttov7pred54L/rtcoef_coriolis_1_windsat.dat")

ssmiRttov.Options.AddInterp = True
ssmiRttov.Options.CO2Data = False
ssmiRttov.Options.VerboseWrapper = True
ssmiRttov.Options.DoCheckinput = False
ssmiRttov.Options.UseQ2m = True
ssmiRttov.Options.ApplyRegLimits = True
ssmiRttov.Options.Verbose = False
ssmiRttov.Options.FastemVersion = 6 

# Load the instruments: for HIRS and MHS do not supply a channel list and
# so read all channels
try:
    ssmiRttov.loadInst(chan_list_ssmi)
except pyrttov.RttovError as e:
    sys.stderr.write("Error loading instrument(s): {!s}".format(e))
    sys.exit(1)

# Load Atlases: (if any)    
ssmiRttov.SurfEmisRefl = np.zeros((2,nprofiles,nchan_ssmi), dtype=np.float64)

# Definition of the observation variables
y_vars = np.array(chan_list_ssmi)

Load successful >>>>> inst_id : 1, nchannels : 14.


In [6]:
# Values for the noise are obtained from HOAPS R matrix (Table 2, p20, HOAPS 4.0 ATBD)

#y_noise = pn.Series(
#    [
#        2.44948974, 1.54919338, 1.341640787, 1.341640787, 1.949358869, 1.341640787
#    ],
#    index=y_vars
#)

#S_y = pn.DataFrame(
#    np.diag(y_noise.values**2),
#    index=y_vars,
#    columns=y_vars,
#)
#

# JUST FOR TESTING THE EFFECT ON SLOPE VIAS IN CROSS-VALIDATION (DON'T USE IN REAL APPLICATION)
# Channels 1-7, 12-16, 23, 24 (MODIFIED values from Deblonde-English 2003) (sigma or std)
#y_noise = pn.Series(
#    [
#        1.3, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 2.0, 1.07, 1.24, 2.5, 1.14, 0.46, 0.47
#   ],
#    index=y_vars
#)
# JUST FOR TESTING THE EFFECT ON SLOPE VIAS IN CROSS-VALIDATION (DON'T USE IN REAL APPLICATION)


# FOR TEST USING NEDT VALUES FROM SSMIS ONLY (From Kunkee 2008)
# Channels 1-7, 12-16, 23, 24 (values in K)
#y_noise = pn.Series(
#    [
#        0.34, 0.32, 0.33, 0.33, 0.34, 0.41, 0.40, 0.33, 0.31, 0.43, 0.25, 0.20, 0.8, 0.9 
#   ],
#    index=y_vars
#)
# FOR TEST USING NEDT VALUES FROM SSMIS ONLY (From Kunkee 2008)


# Channels 1-7, 12-16, 23, 24 (values from Deblonde-English 2003) (sigma or std)
y_noise = pn.Series(
    [
        1.5, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 2.4, 1.27, 1.44, 3.0, 1.34, 0.46, 0.47
   ],
    index=y_vars
)

# Channels 1-7, 13,14,16, 23, 24 (values from Deblonde-English 2003) (sigma or std)
#y_noise = pn.Series(
#    [
#        1.5, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 1.27, 1.44, 1.34, 0.46, 0.47
#    ],
#    index=y_vars
#)

# Channels 1,12-16 (values from Deblonde-English 2003) (sigma or std)
#y_noise = pn.Series(
#    [
#        1.5, 2.4, 1.27, 1.44, 3.0, 1.34
#    ],
#    index=y_vars
#)

# Channels 2-7, 23, 24 (values from Deblonde-English 2003) (sigma or std)
#y_noise = pn.Series(
#    [
#        0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.46, 0.47
#    ],
#    index=y_vars
#)


# Channels 1-16 WindSat (from Table 2 in M. Bettenhausen et al, 
# "A Nonlinear Optimization Algorithm for WindSat Wind Vector Retrievals", 
# IEEE Trans. Geo. Rem. Sen., Vol 44, No. 3, March 2006):

# Take "Polarimetric" channels away:
#y_noise = pn.Series(
#    [
#        0.60, 0.78, 0.69, 0.99, 1.02, 2.02, 1.38,
#        2.51, 1.76, 3.65
#    ],
#    index=y_vars
#)

#y_noise = pn.Series(
#    [
#        0.60, 0.78, 0.69, 0.99, 0.26, 0.09, 1.02, 2.02, 0.28, 0.12, 1.38,
#        2.51, 1.76, 3.65, 0.25, 0.09
#    ],
#    index=y_vars
#)

# Variance values > std**2
S_y = pn.DataFrame(
    np.diag(y_noise.values**2),
    index=y_vars,
    columns=y_vars,
)

In [7]:
# Definition of the indices of the "local prior":
# we will call local prior to the prior built
# using a geographical area "nearby" the observation
# location (i.e. a bounding box around the observation location). 
# All the atmospheric states within such a box 
# Define our a-priori knowledge (in the spatial domain)

lat_min = -90.0
lat_max = -40.0
long_min = -180.0
long_max = 180.0


lat_long_mask = np.array([(prior_all['lat'].values[:]>lat_min)&(prior_all['lat'].values[:]<lat_max)&
                 ((prior_all['long'].values[:]>long_min)&(prior_all['long'].values[:]<long_max))])
lat_long_mask=lat_long_mask.reshape(-1)
#lat_long_mask = ~lat_long_mask


In [ ]:
type(chan_list_ssmi)

In [8]:
# Creating a time index for the local prior:
times1 = prior_all['Time'].values[lat_long_mask]

time_index = np.zeros(len(times1),dtype='datetime64[s]')
for i in range(len(times1)):
    time_index[i] = supporting_routines_m.timestamp2datetime(times1[i])
print(len(times1))    

3650


In [9]:
# Creating lat, long vectors for auxiliary plots e.g. cartopy

lat = prior_all['lat'].values[lat_long_mask]
long = prior_all['long'].values[lat_long_mask]


In [10]:
# Creating a "local" prior: Subset of "prior_all" 
# that contains only the selected lat,long area.

prior_local = supporting_routines_m.maskPrior(prior_all, time_index, lat_long_mask)


In [ ]:
prior_local

In [11]:
# Transform the humidity profiles to a logarithmic scale
# in order to improve their normality (see probability plots)

# IMPORTANT NOTE:
# RTTOV ACCEPTS ONLY kg / kg (LINEAR SCALE)
# THERE IS A LOG2LIN CONVERSION IN THE DEFINITION
# OF THE FORWARD OPERATOR; BUT INSIDE THE OPTIMAL
# ESTIMATION, THE HUMIDITY IS IN LOG SCALE

prior_local['Humidity'] = np.log10(prior_local['Humidity'])

In [ ]:
#def nearest(items, pivot):
#    return min(items, key=lambda x: abs(x - pivot))
len(x_truth.values[:])
#Pressure, press2m, temp2m, hum2m, tskin, salinity
y_obs
ind

In [19]:
indexTotal = np.arange(len(prior_local.time)) # all time indices in prior_local [0,1,...16000,..] 

# Create copy of the indices array
kk = np.copy(indexTotal)

# Shuffle the indices array randomly
np.random.shuffle(kk)

# Split the array in K folds (for K-fold cross validation)
K = 10
hh = np.array_split(kk,K)

rmse = 0.0
mae = 0.0 #np.empty(0)
r2 = 0.0 #np.empty(0)
count = np.empty(0,dtype=int)
wind_opt = np.empty(0)
wind_true = np.empty(0)
wind_opt_err = np.empty(0)

KK = 0

for indexTest in hh: 
    
    KK += 1
# How many "situations" do we want to test 
#(these will be split apart from the a-priori dataset).
# Situation means a (Lat, Long, time) combination:
# i.e. 1 Situation is one datapoint from the prior_local dataset
    nTestSamples = len(indexTest) 

# The actual indices from prior_local  that will be used as testing points: 
# i.e. they are taken out of the a-priori data
#indexTest = np.random.choice(indexTotal,nTestSamples) # This is used only if a Monte Carlo type of cross validation is done

# The test indices are deleted from the a-priori (i.e. training) set
    indexTrain = np.delete(indexTotal,indexTest)

# Split available diverse datasets into:
# - 5 profiles to generate synthetic data (profiles)
# - Rest of the profiles to generate the prior 
# The 5 profiles for synthetic data are EXCLUDED from
# the rest of the datasets (e.g. from the prior)

    profiles = prior_local.isel(time=indexTest)
    prior_local_1 = prior_local.isel(time=indexTrain)
    
    nLev = len(prior_local_1.height)
    # Splitting the priors per season:
    priors, pressure, seasons, months, h_season = supporting_routines_m.priors2seasons(prior_local_1) 
    # Converting priors per season to pandas dataframes:
    #prior_xa, prior_wind = supporting_routines_m.priors2pandas(priors, 
    #                                                            wind_components = True, h_season = h_season)
    
    flavor = 2 # check "priors2Pandas"
    prior_xa, prior_b = supporting_routines_m.priors2Pandas(priors, 
                                                                flavor = flavor, h_season = h_season)    
    x_cov, x_mean = supporting_routines_m.meanCov(prior_xa, seasons)

# Definition of the state variables (names of the variables)
    x_vars = x_mean.state.values

# Assert invertibility of the convariance matrices:
    for season in x_cov.season:
        assert np.linalg.matrix_rank(x_cov.sel(season=season).to_pandas()) ==  x_cov.shape[-1]
        
# Create x_truths: Pandas version of the profiles array (Containing only state variables!)
    x_truths = supporting_routines_m.createTrueState(profiles, flavor=flavor)
# Gather parameters for RTTOV from a-priori knowledge
    
    #prior_b = supporting_routines_m.b_fromPrior(priors)
    
# FOR RTTOV PARAMETERS
# We use pandas tooling (cov, mean) to compute the mean
# and covariance per season:  
    b_cov, b_mean = supporting_routines_m.meanCov(prior_b, seasons)
    b_vars = b_mean.state.values

# Assert invertibility of the convariance matrices:
    for season in x_cov.season:
        assert np.linalg.matrix_rank(b_cov.sel(season=season).to_pandas()) ==  b_cov.shape[-1]   
    
    #w_op = np.zeros(len(indexTest))
    #w_op_err = np.zeros(len(indexTest))
    #w_a = np.zeros(len(indexTest))
    #w_a_err = np.zeros(len(indexTest))
    #w_truth = np.zeros(len(indexTest))
    #chiSquareTest = np.array(len(indexTest), dtype=bool)
    #linearityTest = np.array(len(indexTest), dtype=bool)
    w_op2 = np.empty(0)
    w_op_err2 = np.empty(0)
    w_truth2 = np.empty(0)
    u_op2 = np.empty(0)
    u_op_err2 = np.empty(0)
    u_truth2 = np.empty(0)
    v_op2 = np.empty(0)
    v_op_err2 = np.empty(0)
    v_truth2 = np.empty(0)    
    lat_local = np.empty(0)
    long_local = np.empty(0)
    lat_skip = np.empty(0)
    long_skip = np.empty(0)
    seasonal = np.empty(0,dtype=object)
        
    count_breaks = 0
    count_nc = 0
    for ind in np.arange(len(indexTest)):

        prof = profiles.time.values[ind] 
        seasonIndex = int(np.where(np.isin(months,profiles['time.month'].values[ind]))[0])
        season_prof = seasons[seasonIndex]

# Profile used for generating synthetic observation:
# Contains ONLY the to-be-retrieved parameters: 
# temperature, humidity, 10m surface windspeed
        x_truth = x_truths.iloc[ind]

# Other atmospheric parameters for the syntetic observation 
        #press2m = b_mean.sel(season=season_prof).values[0] 
        #temp2m = b_mean.sel(season=season_prof).values[1] 
# Check this choice for humidity at 2m height (hum2m), TODO *** mario
        #hum2m = 10**(b_mean.sel(season=season_prof).values[2])/1000 
      
        #tskin = b_mean.sel(season=season_prof).values[3] 

        Pressure = pressure.loc[season_prof,:].values[:] 

# Other parameters needed for the radiative transfer model (RTTOV)
        lat1 = np.float(profiles.lat.values[ind])
        long1 = np.float(profiles.long.values[ind])
        datetime_obs = prof
        zenithAngle = 53.0
        salinity = 35

# Create instance of Profiles class; 
# it's a container of the input atmospheric state that RTTOV will simulate

        myProfiles = pyrttov.Profiles(nprofiles, nlevels)

# forward_b_init fills "myProfiles" with the "fixed" parameters for the RTTOV simulation.
# The forward model F(x,b), RTTOV in our case, has two "parameters": x and b
# x is the state vector that is being retrieved (as such it is allowed to change during the retrieval)
# b contains all other parameters that are fixed during the retrieval (everything else that is not being retrieved)

        forward_b_init(Pressure, salinity, 
              lat1, long1, datetime_obs, zenithAngle, myProfiles)
        #forward_b_init(Pressure, press2m, temp2m, hum2m, tskin, salinity, 
        #      lat1, long1, datetime_obs, zenithAngle, myProfiles)
        
# Define a-priori information: mean and covariance matrix
# for a given season (DJF, MAM, JJA, SON)

        x_a = x_mean.sel(season=season_prof).to_pandas()[x_vars]
        S_a = x_cov.sel(season=season_prof).to_pandas().loc[x_vars, x_vars]
        
        # Vector of parameters and its covariance:
        b_p = b_mean.sel(season=season_prof).to_pandas()[b_vars]
        S_b = b_cov.sel(season=season_prof).to_pandas().loc[b_vars, b_vars]
            
# Define dictionary of parameters for the forward model:

        forwardKwArgs = dict(
            myProfiles = myProfiles, 
            ssmiRttov = ssmiRttov
        )

# Call the forward model (i.e. creating synthetic obs.)

        y_obs = forwardRT(x_truth, **forwardKwArgs)
        y_obs = pn.Series(y_obs, index=y_vars)
        assert np.all(np.isfinite(y_obs))
        
        y_K = KRT(x_truth, **forwardKwArgs)

        #print('y_obs done') 
# Create optimalEstimation instance:

        oe_ref = pyOE.optimalEstimation( # oe_1 if windDisambiguation used
            x_vars, # state variable names
            x_a,  # a priori
            S_a, # a priori uncertainty
            y_vars,  # measurement variable names
            y_obs, # observations
            S_y, # observation uncertainty
            forwardRT, # forward Operator
            forwardKwArgs=forwardKwArgs, # additonal function arguments
            x_truth=x_truth, # true profile
            b_vars=b_vars,   # Parameter vector variable names
            b_p=b_p,        # Parameter vector 
            S_b=S_b        # Parameters error covariance matrix 
        )

        #oe_1.doRetrieval()
        oe_ref.doRetrieval()
        
        if not oe_ref.converged :  # oe_1 if windDisambiguation used
            count_nc += 1
            print('NO CONVERGENCE points:')
            print(count_nc)
            lat_skip = np.append(lat_skip,lat1)
            long_skip = np.append(long_skip,long1)
            continue

        #oe_ref = windDisambiguation(oe_1,forwardRT,forwardKwArgs)  
        
# Not Including S_b, b_p:            
        #_, _, w_op1 = supporting_routines_m.splitX(oe_ref.x_op)
        #_, _, w_op_err1 = supporting_routines_m.splitX(oe_ref.x_op_err)
        ##_, _, w_a[ind] = supporting_routines_m.splitX(oe_ref.x_a)
        ##_, _, w_a_err[ind] = supporting_routines_m.splitX(oe_ref.x_a_err)
        #_, _, w_truth1 = supporting_routines_m.splitX(oe_ref.x_truth)
        
    # U & V:
        #_, _, u_op1, v_op1 = supporting_routines_m.splitXUV(oe_ref.x_op)
        #_, _, u_op_err1, v_op_err1 = supporting_routines_m.splitXUV(oe_ref.x_op_err)
        ##_, _, u_a1, v_a1 = supporting_routines_m.splitXUV(oe_ref.x_a)
        ##_, _, u_a_err1, v_a_err1 = supporting_routines_m.splitXUV(oe_ref.x_a)
        #_, _, u_truth1, v_truth1 = supporting_routines_m.splitXUV(oe_ref.x_truth) 
        #u_op2 = np.append(u_op2,u_op1)
        #u_op_err2 = np.append(u_op_err2,u_op_err1)
        #u_truth2 = np.append(u_truth2,u_truth1)
        ## V component
        #v_op2 = np.append(v_op2,v_op1)
        #v_op_err2 = np.append(v_op_err2,v_op_err1)
        #v_truth2 = np.append(v_truth2,v_truth1)          
        ## Compute magnitude using coordinates U and V
        ## w_op1 = np.sqrt( u_op1**2 + v_op1**2 )
        #w_op1, w_op_err1 = supporting_routines_m.UV2Wvar(oe_ref.S_op.loc['00010_nu','00010_nu'],
        #                                          oe_ref.S_op.loc['00010_nv','00010_nv'],
        #                                          oe_ref.S_op.loc['00010_nu','00010_nv'],
        #                                          u_op1,v_op1)
        #w_truth1 = np.sqrt( u_truth1**2 + v_truth1**2 )        
        
# Including S_b, b_p:
       # _,_, w_op1,_,_,_,_ = supporting_routines_m.splitXW_all(oe_ref.x_op)
       # _,_, w_op_err1,_,_,_,_ = supporting_routines_m.splitXW_all(oe_ref.x_op_err)
       ##_,_, w_a,_,_,_,_ = supporting_routines_m.splitXW_all(oe_ref.x_a)
       ##_,_, w_a_err,_,_,_,_ = supporting_routines_m.splitXW_all(oe_ref.x_a_err)
       # _,_, w_truth1,_,_,_,_ = supporting_routines_m.splitXW_all(oe_ref.x_truth)
        
    # U & V:
        _,_,u_op1, v_op1,_,_,_ = supporting_routines_m.splitX_all(oe_ref.x_op)
        _,_,u_op_err1, v_op_err1,_,_,_ = supporting_routines_m.splitX_all(oe_ref.x_op_err)
        #_,_,u_a1, v_a1,_,_,_ = supporting_routines_m.splitX_all(oe_ref.x_a)
        #_,_,u_a_err1, v_a_err1,_,_,_ = supporting_routines_m.splitX_all(oe_ref.x_a_err)
        _,_,u_truth1, v_truth1,_,_,_ = supporting_routines_m.splitX_all(oe_ref.x_truth)
        u_op2 = np.append(u_op2,u_op1)
        u_op_err2 = np.append(u_op_err2,u_op_err1)
        u_truth2 = np.append(u_truth2,u_truth1)
        v_op2 = np.append(v_op2,v_op1)
        v_op_err2 = np.append(v_op_err2,v_op_err1)
        v_truth2 = np.append(v_truth2,v_truth1)          
    # Compute magnitude using coordinates U and V
        w_op1, w_op_err1 = supporting_routines_m.UV2Wvar(oe_ref.S_op.loc['00010_nu','00010_nu'],
                                                  oe_ref.S_op.loc['00010_nv','00010_nv'],
                                                  oe_ref.S_op.loc['00010_nu','00010_nv'],
                                                  u_op1,v_op1)        
        w_truth1 = np.sqrt( u_truth1**2 + v_truth1**2 )   
        
        # For wind components we compare absolute value only
        # SSMIS cannot provide polarimetric measurements
        # so comparing angular information is not a proper metric
        # U component 
        

        w_op2 = np.append(w_op2,w_op1)
        w_op_err2 = np.append(w_op_err2,w_op_err1)
        w_truth2 = np.append(w_truth2,w_truth1)
        
        
        
        seasonal = np.append(seasonal,season_prof)
        
        lat_local = np.append(lat_local,lat1)
        long_local = np.append(long_local,long1)
        
        print('index:')
        print(ind)
        
    
    plotInMap(lat_local, long_local, lat, long, KK, lat_skip, long_skip)
    perSeasonPlots(w_truth2, w_op2, seasonal, KK, windType = 'W10m')
    perSeasonPlots(u_truth2, u_op2, seasonal, KK, windType = 'U10m')
    perSeasonPlots(v_truth2, v_op2, seasonal, KK, windType = 'V10m')
    
    rmse_local, mae_local, r2_local = regressPlots(w_truth2, w_op2, KK, windType = 'W10m')   
    _, _, _ = regressPlots(u_truth2, u_op2, KK, windType = 'U10m')
    _, _, _ = regressPlots(v_truth2, v_op2, KK, windType = 'V10m')
    
    rmse += rmse_local
    mae += mae_local
    r2 += r2_local
    
    #skipRatio = count_nc/(count_nc+len(w_op2)) # ratio of (skipped / total) points in the test dataset
    
    
    
    #if KK == 1: break

rmse = rmse / KK
mae = mae / KK
r2 = r2 / KK


3.01 s, iteration 1, degrees of freedom: 1.73 of 3, converged (x-space):  0.182
4.48 s, iteration 2, degrees of freedom: 1.72 of 3, done.  0.004
index:
0
2.91 s, iteration 1, degrees of freedom: 1.81 of 3, converged (x-space):  0.026
4.34 s, iteration 2, degrees of freedom: 1.82 of 3, done.  0.006
index:
1
2.87 s, iteration 1, degrees of freedom: 1.77 of 3, not converged (x-space):  0.410
4.30 s, iteration 2, degrees of freedom: 1.75 of 3, converged (x-space):  0.001
5.72 s, iteration 3, degrees of freedom: 1.75 of 3, done.  0.000
index:
2


KeyboardInterrupt: 

In [23]:
y_obs

1     218.001508
2     241.558363
3     233.654517
4     224.389229
5     219.285074
6     218.480238
7     218.673964
12    106.142932
13    176.481010
14    187.519452
15    138.060578
16    202.009677
23    235.468802
24    223.442211
dtype: float64

In [ ]:
oe_ref.S_op.loc[['00010_nu','00010_nv'],['00010_nu','00010_nv']]
Uu2 = oe_ref.S_op.loc['00010_nu','00010_nu']  # variance U
Uv2 = oe_ref.S_op.loc['00010_nv','00010_nv']  # variance V 
Uuv = oe_ref.S_op.loc['00010_nu','00010_nv']  # covariance UV

u = np.float64(u_op1.values)
v = np.float64(v_op1.values)
w = np.sqrt(u**2+v**2)

bruteUncertainty = np.sqrt(np.float64(u_op_err1.values)**2+np.float64(v_op_err1.values)**2)

combinedUncertainty = np.sqrt((1/(w**2))*(Uu2*(u)**2+Uv2*(v)**2+2*u*v*Uuv))

print(oe_ref.d_i2)

In [ ]:
#x_vars[0] # state variable names
#type(x_a)  # a priori
#type(S_a), # a priori uncertainty
#y_vars,  # measurement variable names
#y_obs, # observations
#S_y, # observation uncertainty
b_vars,   # Parameter vector variable names
#b_p,        # Parameter vector 
#S_b 

In [ ]:
#u, s, vh = np.linalg.svd(S_a, full_matrices=True)
#plt.scatter(np.arange(len(s)),np.log10(np.absolute(s)/np.absolute(s[0])))
#max_ind = 275
#S_a2 = np.matmul(u[:,0:max_ind],np.matmul(np.diag(s[0:max_ind]),vh[0:max_ind,:]))
#err = np.linalg.norm((S_a2-S_a),ord='fro') / np.linalg.norm((S_a),ord='fro')
#print(err)
#print(np.linalg.cond(S_a))
#print(np.linalg.cond(S_a2))


In [12]:
def windDisambiguation(oe_ref,forwardRT,forwardKwArgs):
    # Procedure proposed in:
    # M. Bettenhausen, et al, "A Nonlinear Optimization Algorithm 
    # for WindSat Wind Vector Retrievals", IEEE Trans. Geo. Rem. Sens.,
    # Vol. 44, No. 3, March 2006
    
    # First retrieved wind
    #_, _, u_op, v_op = supporting_routines_m.splitXUV(oe_ref.x_op)
    _,_,u_op, v_op,_,_,_ = supporting_routines_m.splitX_all(oe_ref.x_op)
    
    # Most of the retrieval parameters are the same as for the original retrieval:
    x_vars = oe_ref.x_vars 
    y_vars = oe_ref.y_vars 
    y_obs = oe_ref.y_obs 
    S_a = oe_ref.S_a
    S_y = oe_ref.S_y
    
    S_b = oe_ref.S_b
    b_vars = oe_ref.b_vars
    b_p = oe_ref.b_p
    
    #forwardRT = oe_ref.forwardRT
    #forwardKwArgs = oe_ref.forwardKwArgs
    x_truth = oe_ref.x_truth 
    
    mini = 100000
    
    rot = np.array([[0,-1],[1,0]]) # Rotation matrix (90 deg rotation)
    # 4 ambiguities (1 per quadrant):
    aprio1 = np.array([[np.float64(u_op.values)],[np.float64(v_op.values)]])   
    aprio2 = np.dot(rot,aprio1)
    aprio3 = np.dot(rot,aprio2)
    aprio4 = np.dot(rot,aprio3)
    
    # Re-use the first retrieved atmospheric state as the new a-priori dataset
    x_a1 = oe_ref.x_op.copy()
    x_a1['00010_nu'] = aprio1[0,0] # a-priori wind 1: as retrieved
    x_a1['00010_nv'] = aprio1[1,0]
    oe_ref1 = pyOE.optimalEstimation(
            x_vars, # state variable names
            x_a1,  # a priori
            S_a, # a priori uncertainty
            y_vars,  # measurement variable names
            y_obs, # observations
            S_y, # observation uncertainty
            forwardRT, # forward Operator
            forwardKwArgs=forwardKwArgs, # additonal function arguments
            x_truth=x_truth, # true profile
            b_vars=b_vars,   # Parameter vector variable names
            b_p=b_p,        # Parameter vector 
            S_b=S_b        # Parameters error covariance matrix 
            )

    oe_ref1.doRetrieval()
    
    # X**2 (eq (2) in reference), goodness of fit of forward model:
    # TODO: S_y should be S_y+S_b' # note Mario    
    numero = np.dot(np.transpose(y_obs-oe_ref1.y_i[-1]),
                    np.dot(np.linalg.inv(oe_ref1.S_y),
                           (y_obs-oe_ref1.y_i[-1])))
    #if oe_ref1.d_i2[-1]<mini:
    if numero < mini:
        mini = numero #oe_ref1.d_i2[-1]
        oe_return = oe_ref1
        index = 1
        
    #_, _,u1, v1 = supporting_routines_m.splitXUV(oe_ref1.x_op)
# 2nd vector
    x_a2 = oe_ref.x_op.copy()
    x_a2['00010_nu'] = aprio2[0,0]
    x_a2['00010_nv'] = aprio2[1,0]
    oe_ref2 = pyOE.optimalEstimation(
            x_vars, # state variable names
            x_a2,  # a priori
            S_a, # a priori uncertainty
            y_vars,  # measurement variable names
            y_obs, # observations
            S_y, # observation uncertainty
            forwardRT, # forward Operator
            forwardKwArgs=forwardKwArgs, # additonal function arguments
            x_truth=x_truth, # true profile
            b_vars=b_vars,   # Parameter vector variable names
            b_p=b_p,        # Parameter vector 
            S_b=S_b        # Parameters error covariance matrix 
            )

    oe_ref2.doRetrieval()
    
    # X**2 (eq (2) in reference), goodness of fit of forward model:
    # TODO: S_y should be S_y+S_b' # note Mario
    numero = np.dot(np.transpose(y_obs-oe_ref2.y_i[-1]),
                    np.dot(np.linalg.inv(oe_ref2.S_y),
                           (y_obs-oe_ref2.y_i[-1])))    
    #if oe_ref2.d_i2[-1]<mini:
    if numero < mini:
        mini = numero #oe_ref2.d_i2[-1]
        oe_return = oe_ref2
        index = 2
        
    #_, _,u2, v2 = supporting_routines_m.splitXUV(oe_ref2.x_op)
# 3rd vector
    x_a3= oe_ref.x_op.copy()
    x_a3['00010_nu'] = aprio3[0,0]
    x_a3['00010_nv'] = aprio3[1,0]
    oe_ref3 = pyOE.optimalEstimation(
            x_vars, # state variable names
            x_a3,  # a priori
            S_a, # a priori uncertainty
            y_vars,  # measurement variable names
            y_obs, # observations
            S_y, # observation uncertainty
            forwardRT, # forward Operator
            forwardKwArgs=forwardKwArgs, # additonal function arguments
            x_truth=x_truth, # true profile
            b_vars=b_vars,   # Parameter vector variable names
            b_p=b_p,        # Parameter vector 
            S_b=S_b        # Parameters error covariance matrix 
            )

    oe_ref3.doRetrieval()
    
    # X**2 (eq (2) in reference), goodness of fit of forward model:
    # TODO: S_y should be S_y+S_b' # note Mario    
    numero = np.dot(np.transpose(y_obs-oe_ref3.y_i[-1]),
                    np.dot(np.linalg.inv(oe_ref3.S_y),
                           (y_obs-oe_ref3.y_i[-1])))     
    
    #if oe_ref3.d_i2[-1]<mini:
    if numero < mini:
        mini = numero #oe_ref3.d_i2[-1]
        oe_return = oe_ref3
        index = 3
        
    #_, _,u3, v3 = supporting_routines_m.splitXUV(oe_ref3.x_op)
# 4th vector
    x_a4 = oe_ref.x_op.copy()
    x_a4['00010_nu'] = aprio4[0,0]
    x_a4['00010_nv'] = aprio4[1,0]
    oe_ref4 = pyOE.optimalEstimation(
            x_vars, # state variable names
            x_a4,  # a priori
            S_a, # a priori uncertainty
            y_vars,  # measurement variable names
            y_obs, # observations
            S_y, # observation uncertainty
            forwardRT, # forward Operator
            forwardKwArgs=forwardKwArgs, # additonal function arguments
            x_truth=x_truth, # true profile
            b_vars=b_vars,   # Parameter vector variable names
            b_p=b_p,        # Parameter vector 
            S_b=S_b        # Parameters error covariance matrix 
            )

    oe_ref4.doRetrieval()
    
    # X**2 (eq (2) in reference), goodness of fit of forward model:
    # TODO: S_y should be S_y+S_b' # note Mario
    numero = np.dot(np.transpose(y_obs-oe_ref4.y_i[-1]),
                    np.dot(np.linalg.inv(oe_ref4.S_y),
                           (y_obs-oe_ref4.y_i[-1])))     
    
    #if oe_ref4.d_i2[-1]<mini:
    if numero < mini:
        mini = numero #oe_ref4.d_i2[-1]
        oe_return = oe_ref4
        index = 4
        
    #_, _,u4, v4 = supporting_routines_m.splitXUV(oe_ref4.x_op)
    print(index)

    return oe_return

In [ ]:
#f, ax = plt.subplots(figsize=(9, 6))
#sns.heatmap(S_a, ax=ax, cbar_kws={'label':'Covariance [K$^2$]'})
#ax.set_title('$\mathbb{S}_{a}$ Temperature')
#plt.savefig('Test_Sa.png', dpi=300)

In [13]:
def forwardRT(X, myProfiles, ssmiRttov):
    
    # TODO: Add assertions, tests *** mario

    # X contains T, Q and W10, lets split the vector
    #temperature, humidity, wind10m = supporting_routines_m.splitX(X)
    
    # if wind speed in components:
    #temperature, humidity, u10m, v10m = supporting_routines_m.splitXUV(X)
    temperature, humidity, u10m, v10m, bp2m, bt2m, btsk = supporting_routines_m.splitX_all(X)
    #temperature, humidity, wind10m, bp2m, bt2m, btsk = supporting_routines_m.splitXW_all(X)
    
    # humdity is in log10 scale, convert to linear in kg/kg
    humidity = (10**humidity) / 1000.
    # or abs_humidity? *** note mario
    
    #hum2m = 10**(np.float64(bh2m.values))/1000 
    
    myProfiles.T = expand2nprofiles(np.array(temperature.values,dtype=np.float64), nprofiles)
    myProfiles.Q = expand2nprofiles(np.array(humidity.values,dtype=np.float64), nprofiles)  

    myProfiles.S2m[0,0] = np.float64(bp2m.values) # surface pressure
    myProfiles.S2m[0,1] = np.float64(bt2m.values) # 2m temperature
    #myProfiles.S2m[0,2] = hum2m                   # 2m humidity, is not mandatory for RTTOV
    myProfiles.Skin[0,0] = np.float64(btsk.values)
    
    #myProfiles.S2m[0,3] = np.float64(wind10m.values) / ( np.sqrt(2) ) #  10m windspeed, u component
    #myProfiles.S2m[0,4] = np.float64(wind10m.values) / ( np.sqrt(2) ) #  10m windspeed, v component
    
    # if wind in components:
    myProfiles.S2m[0,3] = np.float64(u10m.values) #  10m windspeed, u component
    myProfiles.S2m[0,4] = np.float64(v10m.values) #  10m windspeed, v component  
    
    
    
    
    ssmiRttov.Profiles = myProfiles
    
    ssmiRttov.SurfEmisRefl[:,:,:] = -1. # need to "reset" to -1 every time RTTOV is called; 
    # -1 indicates to RTTOV to use internal values for surface emissivity.
    
    try:
        ssmiRttov.runDirect()
    except pyrttov.RttovError as e:
        sys.stderr.write("Error running RTTOV direct model: {!s}".format(e))
        sys.exit(1)    
        
    TB = ssmiRttov.BtRefl[0, :]
    
    return TB

In [14]:
def KRT(X, myProfiles, ssmiRttov):
    
    # TODO: Add assertions, tests *** mario

    # X contains T, Q and W10, lets split the vector
    #temperature, humidity, wind10m = supporting_routines_m.splitX(X)
    
    # if wind speed in components:
    #temperature, humidity, u10m, v10m = supporting_routines_m.splitXUV(X)
    temperature, humidity, u10m, v10m, bp2m, bt2m, btsk = supporting_routines_m.splitX_all(X)
    #temperature, humidity, wind10m, bp2m, bt2m, btsk = supporting_routines_m.splitXW_all(X)
    
    # humdity is in log10 scale, convert to linear in kg/kg
    humidity = (10**humidity) / 1000.
    # or abs_humidity? *** note mario
    
    #hum2m = 10**(np.float64(bh2m.values))/1000 
    
    myProfiles.T = expand2nprofiles(np.array(temperature.values,dtype=np.float64), nprofiles)
    myProfiles.Q = expand2nprofiles(np.array(humidity.values,dtype=np.float64), nprofiles)  

    myProfiles.S2m[0,0] = np.float64(bp2m.values) # surface pressure
    myProfiles.S2m[0,1] = np.float64(bt2m.values) # 2m temperature
    #myProfiles.S2m[0,2] = hum2m                   # 2m humidity, is not mandatory for RTTOV
    myProfiles.Skin[0,0] = np.float64(btsk.values)
    
    #myProfiles.S2m[0,3] = np.float64(wind10m.values) / ( np.sqrt(2) ) #  10m windspeed, u component
    #myProfiles.S2m[0,4] = np.float64(wind10m.values) / ( np.sqrt(2) ) #  10m windspeed, v component
    
    # if wind in components:
    myProfiles.S2m[0,3] = np.float64(u10m.values) #  10m windspeed, u component
    myProfiles.S2m[0,4] = np.float64(v10m.values) #  10m windspeed, v component  
    
    
    
    
    ssmiRttov.Profiles = myProfiles
    
    ssmiRttov.SurfEmisRefl[:,:,:] = -1. # need to "reset" to -1 every time RTTOV is called; 
    # -1 indicates to RTTOV to use internal values for surface emissivity.
    
    try:
        ssmiRttov.runK()
    except pyrttov.RttovError as e:
        sys.stderr.write("Error running RTTOV K model: {!s}".format(e))
        sys.exit(1)    
        
    TB = ssmiRttov.BtRefl[:, :]
    
    return TB

In [15]:
def expand2nprofiles(n, nprof):
    # Transform 1D array to a [nprof, nlevels] array
    outp = np.empty((nprof, len(n)), dtype=n.dtype)
    for i in range(nprof):
        outp[i, :] = n[:]
    return outp


def forward_b_init(pressure, salinity, lat, long, datetime_obs64, 
                   zenithAngle, myProfiles):
#def forward_b_init(pressure, press2m, temp2m, hum2m, tskin, salinity, 
#              lat, long, datetime_obs64, zenithAngle, myProfiles):
    
    
    nprofiles = 1 # Hardcoded for now; to be used in a retrieval framework (e.g.
    #  analyze 1 profile at the time); bad choice, to change. TODO *** mario  
    
    # The rest of the code uses datetime64 format (numpy), but I have to pass the obs date as integers to RTTOV
    datetime_obs = supporting_routines_m.datetime64_to_datetime(datetime_obs64)
    
    
    # 10m wind speed is an unknown parameter to be retrieved (is part of x not of b):
    s2m = np.array([[0., 0., 0., 0., 0., 0.]], dtype=np.float64)
    #s2m = np.array([[press2m, temp2m, hum2m, 0., 0., 0.]], dtype=np.float64)
    
    angles = np.array([[zenithAngle,  0., 0., 0.]], dtype=np.float64)
    
    skin = np.array([[0., salinity, 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=np.float64)
    #skin = np.array([[tskin, salinity, 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=np.float64)
    
    surftype = np.array([[1, 1]], dtype=np.int32) # [sea, ocean] Harcoded for now, TODO *** mario
    
    surfgeom = np.array([[lat, long, 0.]], dtype=np.float64) # elevation harcoded to 0 for now, TODO *** mario
    
    date_times = np.array([[datetime_obs.year, datetime_obs.month, datetime_obs.day,
                            datetime_obs.hour, datetime_obs.minute, datetime_obs.second]], dtype=np.int32)
    
    
    myProfiles.GasUnits = 1  # kg/kg (see RTTOV doc. for other options) # Harcoded for now, TODO *** mario
    myProfiles.P = expand2nprofiles(pressure, nprofiles)
    myProfiles.S2m = s2m
    myProfiles.Angles = angles
    myProfiles.Skin = skin
    myProfiles.SurfType = surftype
    myProfiles.SurfGeom = surfgeom
    myProfiles.DateTimes = date_times
    
    #return myProfiles

In [16]:
def plotInMap(lat2, long2, lat, long, indice, lat_s = None, long_s = None):
    
    #lat2 = profiles.lat.values
    #long2 = profiles.long.values

    plt.figure(figsize=(13,6.2))
    
    ax = plt.subplot(111, projection=ccrs.PlateCarree())

    ax.scatter(long,\
                   lat,marker='.',color='red')
    ax.scatter(long2,\
                   lat2,marker='o',color='blue')
    if (lat_s is not None)and(long_s is not None):
        ax.scatter(long_s,\
                   lat_s,marker='o',color='yellow')
                
    ax.coastlines();

    ax.grid(True)

    ax.set_xlabel('Longitude [deg]')
    ax.set_ylabel('Latitude [deg]')

    ax.stock_img();

    gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=2, 
                      color='black', alpha=0.5, linestyle='--', draw_labels=True)
    gl.xlabels_top = False
    gl.ylabels_left = False
    gl.ylabels_right=True
    gl.xlines = True
    gl.xlocator = mticker.FixedLocator([-180, -120, -60, 0, 60, 120, 180])
    gl.ylocator = mticker.FixedLocator([-60, -30, 0, 30, 60])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER

    plt.savefig('Loc_Fold_'+str(indice)+'.png', dpi=300)

In [17]:
def perSeasonPlots(truth,optimized,seasonal, indice, windType = 'W10m'):
    
    colors = {'DJF':'blue', 'MAM':'green', 'JJA':'orange', 'SON':'purple'}
    
    df = pn.DataFrame(dict(Truth=truth, Optimized=optimized, Season = seasonal))
    
    seasonGroup = df.groupby('Season')

    for name, group in seasonGroup:
        #print name
        #print group 
        
        w_truth = group['Truth'].values[:]
        w_opt = group['Optimized'].values[:]

        error_local = w_truth - w_opt
        mean_error_local = np.mean( error_local )
        median_error_local = np.median( error_local )
        multiplicativeBias = np.mean(w_truth) / np.mean(w_opt)

        rmse_local = np.sqrt(mean_squared_error(w_truth, w_opt)) 
        mae_local = median_absolute_error(w_truth, w_opt)
        r2_local = r2_score(w_truth, w_opt)

        
        wt = np.reshape( w_truth, (len(w_truth),1) ) # for using with .fit below
        
        # Ordinary least squares:
        regr = LinearRegression()
    
        # Robust fit with Ransac algo. (sklearn lib.)
        ransac = RANSACRegressor()
        
        # HuberRegressor
        #huber = HuberRegressor()

        # Train the model using the training sets
        
            # OLS fit:
        regr.fit(wt, w_opt)
        # Ransac fit:
        ransac.fit(wt, w_opt)
        # Huber fit
        #huber.fit(wt, w_opt)
        
        # mask inlier-outlier from Ransac:
        inlier_mask = ransac.inlier_mask_
        outlier_mask = np.logical_not(inlier_mask)
        
        # mask inlier-outlier from Huber:     
        #outlier_mask = huber.outliers_
        #inlier_mask = np.logical_not(outlier_mask)
    
        # Make predictions using the testing set
        # OLS:
        wind_pred = regr.predict(wt)  
        # Ransac:
        wind_ransac = ransac.predict(wt)
        # Huber:
        #wind_huber = huber.predict(wt)   
    
        fig, (ax1,ax2) = plt.subplots(ncols=2, sharey = True)
        # Plot outputs
        ax1.scatter(w_truth, w_opt,  c=group['Season'].map(colors), alpha= 0.5, label = 'Optimal vs True')
        ax1.plot(w_truth, wind_pred, color='black', linewidth=1.5, label = 'Linear regression')
        ax1.plot(w_truth, w_truth, color='gray', linewidth=1.5, label = 'Slope 1 line')

        ax1.set_ylabel('NS Wind Speed [m/s], Retrieved')
        ax1.set_xlabel('NS Wind Speed [m/s], True')
        ax1.grid(True)
        ax1.set_title('OLS '+windType)
        ax1.text(
            0.95,
            0.05,
            '%s\nm = %.3g\nb = %.3g%s\nrmse = %.3g%s\nmae = %.3g%s\nr$^2$ = %.3g' % (
                '$mx+b$',
                np.float(regr.coef_),
                np.float(regr.intercept_),'m/s',
                rmse_local,'m/s',
                mae_local,'m/s',
                r2_local),
            horizontalalignment='right',
            verticalalignment='bottom',
            transform=ax1.transAxes)
        
        # Plot Ransac
        ax2.scatter(w_truth[inlier_mask], w_opt[inlier_mask],  
                c=group['Season'][inlier_mask].map(colors), alpha = 0.5, label = 'Retrieved')
        ax2.scatter(w_truth[outlier_mask], w_opt[outlier_mask],  
                color='magenta', alpha = 0.4, label = 'Outlier')
        ax2.plot(w_truth, wind_ransac, color='black', linewidth=1.5, label = 'Regression')
        ax2.plot(w_truth, w_truth, color='gray', linewidth=1.5, label = 'm = 1')
    
        ax2.legend()
        ax2.set_xlabel('NS Wind Speed [m/s], True')
        ax2.grid(True)
        ax2.set_title('Ransac '+windType)
        ax2.text(
            0.95,
            0.05,
            '%s\nm = %.3g\nb = %.3g%s\nrmse = %.3g%s\nmae = %.3g%s\nr$^2$ = %.3g' % (
                '$mx+b$',
                np.float(ransac.estimator_.coef_),
                np.float(ransac.estimator_.intercept_),'m/s',
                np.sqrt(mean_squared_error(w_truth[inlier_mask], w_opt[inlier_mask])) ,'m/s',
                median_absolute_error(w_truth[inlier_mask], w_opt[inlier_mask]),'m/s',
                np.float(r2_score(w_truth[inlier_mask], w_opt[inlier_mask]))),
            horizontalalignment='right',
            verticalalignment='bottom',
            transform=ax2.transAxes)

        plt.legend(loc=(0.02,0.73), prop={'size': 9}) 
        plt.savefig('seas_wind'+name+windType+'_'+str(indice), dpi=300)  
        
        # Histogram
        bins_all = 5

        fig, ax = plt.subplots(1)
        ax.hist(error_local, bins = bins_all, label=name)
        ax.legend()
        ax.text(0.95,0.75,
                'Mean error = %.3g\nMedian error = %.3g\nBias (M) = %.3g' % ( 
                    mean_error_local, 
                    median_error_local,
                    multiplicativeBias),
                horizontalalignment='right',
                verticalalignment='top',
                transform=ax.transAxes)
    
        ax.grid(True)
        plt.savefig('seas_err'+name+windType+'_'+str(indice), dpi=300)
    


In [18]:
def regressPlots(truth, optimized, indice, windType = 'Wind'):
    
        #error_local = truth - optimized
        #mean_error_local = np.mean( error_local )
        #median_error_local = np.median( error_local )
    
        rmse_local = np.sqrt(mean_squared_error(truth, optimized)) 
        mae_local = median_absolute_error(truth, optimized)
        r2_local = r2_score(truth, optimized)

    
        # Ordinary least squares:
        regr = LinearRegression()
    
        # Robust fit with Ransac algo. (sklearn lib.)
        ransac = RANSACRegressor()
    
        # Train the model using the training sets
        wt = np.reshape(truth,(len(truth),1)) # for using with .fit below
    
        # OLS fit:
        regr.fit(wt, optimized)
        # Ransac fit:
        ransac.fit(wt, optimized)
        # mask inlier-outlier from Ransac:
        inlier_mask = ransac.inlier_mask_
        outlier_mask = np.logical_not(inlier_mask)
    
        # Make predictions using the testing set
        # OLS:
        wind_pred = regr.predict(wt)  
        # Ransac:
        wind_ransac = ransac.predict(wt)
    
        fig, (ax1,ax2) = plt.subplots(ncols=2, sharey = True)
        # Plot outputs
        ax1.scatter(truth, optimized,  color='dodgerblue', alpha = 0.5, label = 'Optimal vs True')
        ax1.plot(truth, wind_pred, color='black', linewidth=1.5, label = 'Linear regression')
        ax1.plot(truth, truth, color='gray', linewidth=1.5, label = 'Slope 1 line')

        ax1.set_ylabel('NS Wind Speed [m/s], Retrieved')
        ax1.set_xlabel('NS Wind Speed [m/s], True')
        ax1.grid(True)
        ax1.set_title('OLS '+windType)
        ax1.text(
            0.95,
            0.05,
            '%s\nm = %.3g\nb = %.3g%s\nrmse = %.3g%s\nmae = %.3g%s\nr$^2$ = %.3g' % (
            '$mx+b$',
            np.float(regr.coef_),
            np.float(regr.intercept_),'m/s',
            rmse_local,'m/s',
            mae_local,'m/s',
            r2_local),
            horizontalalignment='right',
            verticalalignment='bottom',
            transform=ax1.transAxes)

        # Plot outputs
        ax2.scatter(truth[inlier_mask], optimized[inlier_mask],  
                color='dodgerblue', alpha = 0.5, label = 'Retrieved')
        ax2.scatter(truth[outlier_mask], optimized[outlier_mask],  
                color='magenta', alpha = 0.5, label = 'Outlier')
        ax2.plot(truth, wind_ransac, color='black', linewidth=1.5, label = 'Regression')
        ax2.plot(truth, truth, color='gray', linewidth=1.5, label = 'm = 1')
    
        ax2.legend()
        ax2.set_xlabel('NS Wind Speed [m/s], True')
        ax2.grid(True)
        ax2.set_title('Ransac '+windType)
        ax2.text(
            0.95,
            0.05,
            '%s\nm = %.3g\nb = %.3g%s\nrmse = %.3g%s\nmae = %.3g%s\nr$^2$ = %.3g' % (
            '$mx+b$',
            np.float(ransac.estimator_.coef_),
            np.float(ransac.estimator_.intercept_),'m/s',
            np.sqrt(mean_squared_error(truth[inlier_mask], optimized[inlier_mask])) ,'m/s',
            median_absolute_error(truth[inlier_mask], optimized[inlier_mask]),'m/s',
            np.float(r2_score(truth[inlier_mask], optimized[inlier_mask]))),
            horizontalalignment='right',
            verticalalignment='bottom',
            transform=ax2.transAxes)

        plt.legend(loc=(0.02,0.73), prop={'size': 9}) 
        plt.savefig('windR_'+windType+str(indice), dpi=300)
        
        return rmse_local, mae_local, r2_local



In [ ]:
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(
    S_y.where(S_y > 0), 
    annot=True, 
    fmt='.2f', 
    ax=ax, 
    linewidths=.05, 
    cbar_kws={'label':'Covariance [K$^2$]'}
)
ax.set_title(r'Measurement uncertainty $\mathbf{S}_{y}$')
plt.savefig('Sy_1-7_12-16_23-24', dpi=450)

In [ ]:

# Here inner loop should start:

w_op = np.zeros(len(indexTest))
w_op_err = np.zeros(len(indexTest))
w_a = np.zeros(len(indexTest))
w_a_err = np.zeros(len(indexTest))
w_truth = np.zeros(len(indexTest))
chiSquareTest = np.array(len(indexTest), dtype=bool)
linearityTest = np.array(len(indexTest), dtype=bool)

for ind in np.arange(len(indexTest)):

    prof = profiles.time.values[ind] 
    seasonIndex = int(np.where(np.isin(months,profiles['time.month'].values[ind]))[0])
    season_prof = seasons[seasonIndex]

# Profile used for generating synthetic observation:
# Contains ONLY the to-be-retrieved parameters: 
# temperature, humidity, 10m surface windspeed
    x_truth = x_truths.iloc[ind]

# Other atmospheric parameters for the syntetic observation 
    press2m = b_mean.sel(season=season_prof).values[0] 
    temp2m = b_mean.sel(season=season_prof).values[1] 
# Check this choice for humidity at 2m height (hum2m), TODO *** mario
    hum2m = 10**(b_mean.sel(season=season_prof).values[2])/1000 
    tskin = b_mean.sel(season=season_prof).values[3] 

    Pressure = pressure.loc[season_prof,:].values[:] 

# Other parameters needed for the radiative transfer model (RTTOV)
    lat1 = np.float(profiles.lat.values[ind])
    long1 = np.float(profiles.long.values[ind])
    datetime_obs = prof
    zenithAngle = 53.0
    salinity = 35

# Create instance of Profiles class; 
# it's a container of the input atmospheric state that RTTOV will simulate

    myProfiles = pyrttov.Profiles(nprofiles, nlevels)

# forward_b_init fills "myProfiles" with the "fixed" parameters for the RTTOV simulation.
# The forward model F(x,b), RTTOV in our case, has two "parameters": x and b
# x is the state vector that is being retrieved (as such it is allowed to change during the retrieval)
# b contains all other parameters that are fixed during the retrieval (everything else that is not being retrieved)

    forward_b_init(Pressure, press2m, temp2m, hum2m, tskin, salinity, 
              lat1, long1, datetime_obs, zenithAngle, 1, nlevels, myProfiles)

# Define a-priori information: mean and covariance matrix
# for a given season (DJF, MAM, JJA, SON)

    x_a = x_mean.sel(season=season_prof).to_pandas()[x_vars]
    S_a = x_cov.sel(season=season_prof).to_pandas().loc[x_vars, x_vars]

# Define dictionary of parameters for the forward model:

    forwardKwArgs = dict(
        myProfiles = myProfiles, 
        ssmiRttov = ssmiRttov
    )

# Call the forward model (i.e. creating synthetic obs.)

    y_obs = forwardRT(x_truth, **forwardKwArgs)
    y_obs = pn.Series(y_obs, index=y_vars)
    assert np.all(np.isfinite(y_obs))

# Create optimalEstimation instance:

    oe_ref = pyOE.optimalEstimation(
        x_vars, # state variable names
        x_a,  # a priori
        S_a, # a priori uncertainty
        y_vars,  # measurement variable names
        y_obs, # observations
        S_y, # observation uncertainty
        forwardRT, # forward Operator
        forwardKwArgs=forwardKwArgs, # additonal function arguments
        x_truth=x_truth, # true profile
    )

    oe_ref.doRetrieval()

    _, _, w_op[ind] = supporting_routines_m.splitX(oe_ref.x_op)
    _, _, w_op_err[ind] = supporting_routines_m.splitX(oe_ref.x_op_err)
    #_, _, w_a[ind] = supporting_routines_m.splitX(oe_ref.x_a)
    #_, _, w_a_err[ind] = supporting_routines_m.splitX(oe_ref.x_a_err)
    _, _, w_truth[ind] = supporting_routines_m.splitX(oe_ref.x_truth)
    
    rmse = np.sqrt(mean_squared_error(w_truth, w_op))
    mae = median_absolut_error(w_truth, w_op)
    
    #chiSquareTest[ind] = oe_ref.chiSquareTest(significance=0.05)
    #for x in chiSquareTest[0]: print (x)
    #linearity, trueLinearityChi2, trueLinearityChi2Crit = oe_ref.linearityTest(significance=0.05)
    #boole = np.bool(trueLinearityChi2 < trueLinearityChi2Crit)
    #linearityTest[ind] = boole
    


In [ ]:
print('All units below are m/s:')
print('Optimal wind speed:')
print(w_op)
print('Optimal wind speed error:')
print(w_op_err)
print('A-priori wind speed:')
print(w_a)
print('A-priori wind speed error:')
print(w_a_err)
print('True wind speed:')
print(w_truth)

In [ ]:
# ploting graph 
fig, ax = plt.subplots(1)

plt.scatter(w_truth, w_op, 
             marker ='o', color='orange', label='D') 



In [ ]:

rmse = np.sqrt(mean_squared_error(w_truth, w_op))

print(rmse)

In [ ]:
t_op, q_op, w_op = supporting_routines_m.splitX(oe_ref.x_op)
t_op_err, q_op_err, w_op_err = supporting_routines_m.splitX(oe_ref.x_op_err)
t_a, q_a, w_a = supporting_routines_m.splitX(oe_ref.x_a)
t_a_err, q_a_err, w_a_err = supporting_routines_m.splitX(oe_ref.x_a_err)
t_truth, q_truth, w_truth = supporting_routines_m.splitX(oe_ref.x_truth)
print('All units below are m/s:')
print('Optimal wind speed:')
print(w_op)
print('Optimal wind speed error:')
print(w_op_err)
print('A-priori wind speed:')
print(w_a)
print('A-priori wind speed error:')
print(w_a_err)
print('True wind speed:')
print(w_truth)

In [ ]:
fig = supporting_routines_m.plotMwrResultsX(
    oe_ref,
    h=pressure.loc[season],
    hlabel='Pressure [hPa]',
    title='Reference run')
plt.savefig('TQ_with_1Sigma_1', dpi=450)

In [ ]:
# Map related plots (Cartopy)

#lat2 = [-53.06520080566406,-41.81959915161133, -31.13629913330078,-47.16130065917969,-36.618499755859375]
#long2 = [105.07499694824219,12.777799606323242,40.15999984741211,102.4000015258789,134.5]

lat2 = profiles.lat.values
long2 = profiles.long.values

plt.figure(figsize=(13,6.2))
    
ax = plt.subplot(111, projection=ccrs.PlateCarree())

ax.scatter(long,\
                   lat,marker='.',color='red')
ax.scatter(long2,\
                   lat2,marker='o',color='blue')
ax.coastlines();

ax.grid(True)

ax.set_xlabel('Longitude [deg]')
ax.set_ylabel('Latitude [deg]')

ax.stock_img();

gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=2, color='black', alpha=0.5, linestyle='--', draw_labels=True)
gl.xlabels_top = False
gl.ylabels_left = False
gl.ylabels_right=True
gl.xlines = True
gl.xlocator = mticker.FixedLocator([-180, -120, -60, 0, 60, 120, 180])
gl.ylocator = mticker.FixedLocator([-60, -30, 0, 30, 60])
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

plt.savefig('Sampling_map.png', dpi=450)

In [ ]:
# Create covariance matrix as a pandas dataframe:
S_a = x_cov.sel(season='MAM').to_pandas()

f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(S_a.iloc[2::2, 2::2], ax=ax, cbar_kws={'label':'Covariance [K$^2$]'})
ax.set_title('$\mathbb{S}_{a}$ Temperature')
plt.savefig('SaTemp_LatLong_0_60_-60_0.png', dpi=450)

f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(S_a.iloc[1::2, 1::2], ax=ax, cbar_kws={'label':'Covariance [log$_{10}$(g/kg)$^2$]'})
ax.set_title('$\mathbb{S}_{a}$ Humidity')
plt.savefig('SaHum_LatLong_0_60_-60_0.png', dpi=450)

f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(S_a.iloc[::10, ::10], ax=ax, cbar_kws={'label':'Covariance [log$_{10}$(g/kg)$^2$]'})
ax.set_title('$\mathbb{S}_{a}$ Wind')
plt.savefig('SaHum_LatLong_0_60_-60_0.png', dpi=450)

In [ ]:
dtime = datetime(year=1990,month=10,day=2,hour=18,minute=30,second=30)
type(dtime.year)

In [ ]:
for var in ['Humidity', 'Temperature']:
    for h in range(0, 137, 20):
        fig, (ax, ax1) = plt.subplots(ncols=2)
        plt.suptitle('%s at level %i' % (var,h))
        (osm, osr), (slope, intercept, r) = stats.probplot(
            prior_all[var].isel(height=h).values[lat_long_mask],
            dist="norm",
            plot=ax,
            fit=True,
            rvalue=False)
        ax.text(
            0.05,
            0.95,
            '%s) %s, R$^2$=%.3g' % ('a', var, r**2),
            horizontalalignment='left',
            verticalalignment='top',
            transform=ax.transAxes)

        (osm, osr), (slope, intercept, r) = stats.probplot(
            np.log10(prior_all[var].isel(height=h).values[lat_long_mask]),
            dist="norm",
            plot=ax1,
            fit=True,
            rvalue=False)
        ax1.text(
            0.05,
            0.95,
            '%s) %s, R$^2$=%.3g' % ('b', 'log10(%s)'%var, r**2),
            horizontalalignment='left',
            verticalalignment='top',
            transform=ax1.transAxes)
        fig.tight_layout()
        plt.savefig('Prob_plot'+var+str(h)+'.png', dpi=450) 

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)
for season in ['all'] + seasons:
    t, q, w = supporting_routines_m.splitX(x_mean.sel(season=season).to_pandas())
    ax1.plot(t, t.index, label=season)
    ax2.plot(q, q.index, label=season)

ax2.legend()
ax1.set_xlabel('Temperature [K]')
ax2.set_xlabel('Specific Humidity [log$_{10}$(g/kg)]')
ax1.set_ylabel('Height [m]')

ax1.grid(True)
ax2.grid(True)

plt.savefig('LatLong-70_-20_0_180.png', dpi=450)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)
for season in ['all'] + seasons:
    # mean and (cov)-variances
    x_a = x_mean.sel(season=season).to_pandas()[x_vars]
    S_a = x_cov.sel(season=season).to_pandas().loc[x_vars, x_vars]
    
    xa_t, xa_q = supporting_routines.splitTQ(x_a)
    # Errors:
    xa_err = np.sqrt(pn.Series(np.diag(S_a), index=list(x_vars)))
    xa_err_t, xa_err_q = supporting_routines.splitTQ(xa_err)
    
    
    ax1.plot(xa_t, xa_t.index,label=season) #, color='C1', label='Prior')
    ax1.fill_betweenx(xa_t.index, xa_t+xa_err_t, xa_t-xa_err_t,
                          alpha=0.2)
                          #color='C1', alpha=0.2)
    ax2.plot(xa_q, xa_q.index,label=season) #, color='C0', label='Prior')
    ax2.fill_betweenx(xa_q.index, xa_q+xa_err_q, xa_q-xa_err_q,
                          alpha=0.2)
                          #color='C0', alpha=0.2)
    #t, q = supporting_routines.splitTQ(x_mean.sel(season=season).to_pandas())
    #ax1.plot(t, t.index, label=season)
    #ax2.plot(q, q.index, label=season)

ax2.legend()
ax1.set_xlabel('Temperature [K]')
ax2.set_xlabel('Specific Humidity [log$_{10}$(g/kg)]')
ax1.set_ylabel('Height [m]')

ax1.grid(True)
ax2.grid(True)

plt.savefig('LatLong_-70_-20_0_180_meanErr.png', dpi=450)

In [ ]:
for var in ['W10']:
    fig, (ax, ax1) = plt.subplots(ncols=2)
    plt.suptitle('%s on surface' % (var))
    (osm, osr), (slope, intercept, r) = stats.probplot(
        prior_all[var].values[lat_long_mask],
        dist="norm",
        plot=ax,
        fit=True,
        rvalue=False)
    ax.text(
        0.05,
        0.95,
        '%s) %s, R$^2$=%.3g' % ('a', var, r**2),
        horizontalalignment='left',
        verticalalignment='top',
        transform=ax.transAxes)

    (osm, osr), (slope, intercept, r) = stats.probplot(
        np.log10(prior_all[var].values[lat_long_mask]),
        dist="norm",
        plot=ax1,
        fit=True,
        rvalue=False)
    ax1.text(
        0.05,
        0.95,
        '%s) %s, R$^2$=%.3g' % ('b', 'log10(%s)'%var, r**2),
        horizontalalignment='left',
        verticalalignment='top',
        transform=ax1.transAxes)
    fig.tight_layout()

In [ ]:
from sklearn import datasets


# Load the diabetes dataset
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)

# Use only one feature
diabetes_X = diabetes_X[:, np.newaxis, 2]

# Split the data into training/testing sets
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]

# Split the targets into training/testing sets
diabetes_y_train = diabetes_y[:-20]
diabetes_y_test = diabetes_y[-20:]

# Create linear regression object
regr = LinearRegression()

# Train the model using the training sets
regr.fit(diabetes_X_train, diabetes_y_train)

# Make predictions using the testing set
diabetes_y_pred = regr.predict(diabetes_X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(diabetes_y_test, diabetes_y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(diabetes_y_test, diabetes_y_pred))

# Plot outputs
plt.scatter(diabetes_X_test, diabetes_y_test,  color='black')
plt.plot(diabetes_X_test, diabetes_y_pred, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()


In [ ]:
diabetes_X_train.shape
diabetes_y_train.shape

In [ ]:
print(w_truth2.shape) 
print(w_op2.shape)
#w = np.reshape(w_truth2,(len(w_truth2),1))
w = np.copy(w_truth2)
w.reshape(-1,1)
print(w.shape)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)

t, q, w = supporting_routines_m.splitX(oe_ref.x_op)
ax1.plot(t, t.index, label='DJF op')
ax2.plot(q, q.index, label='DJF op')

t, q, w = supporting_routines_m.splitX(oe_ref.x_a)
ax1.plot(t, t.index, label='DJF a-priori')
ax2.plot(q, q.index, label='DJF a-priori')

t, q, w = supporting_routines_m.splitX(oe_ref.x_truth)
ax1.plot(t, t.index, label='DJF truth')
ax2.plot(q, q.index, label='DJF truth')

ax2.legend()
ax1.set_xlabel('Temperature [K]')
ax2.set_xlabel('Specific Humidity [log$_{10}$(g/kg)]')
ax1.set_ylabel('Height [m]')

ax1.grid(True)
ax2.grid(True)

plt.savefig('TQ_true_vs_other_1', dpi=450)

In [ ]:
# making a simple plot for winds 

x =[1, 2, 3, 4, 5] 
x_season =[1, 2, 3, 4] 
# A-priori:
y_a =[8.73, 9.09, 9.09, 8.85, 9.58]
y_a_season =[8.73, 9.09, 8.85, 9.58]
# Optimal:
y_op1 = [12.97, 6.07, 17.72, 12.51, 10.81]
y_op2 = [12.16, 6.03, 16.09, 12.04, 10.25]
y_op3 = [12.76, 6.68, 16.60, 11.74, 11.42]
y_op4 = [9.77, 5.77, 14.79, 10.79, 8.79]

# True:
y_true1 = [13.69, 6.02, 20.59, 12.90, 11.72]
 
# creating error 
y_a_error = [4.12, 4.90, 4.90, 3.79, 4.31]
y_a_error_season = [4.12, 4.90, 3.79, 4.31]
y_op_error1 = [1.67, 1.73, 2.81, 1.82, 1.78]
y_op_error2 = [2.37, 2.29, 3.48, 2.39, 2.39]
y_op_error3 = [1.82, 1.93, 3.05, 1.95, 1.95]
y_op_error4 = [3.72, 3.85, 3.93, 3.34, 3.76]

# ploting graph 
fig, ax = plt.subplots(1)

plt.errorbar(x_season, y_a_season, 
             yerr = y_a_error_season, 
             fmt ='o', capsize=8, color='orange', label='A-priori NS wind speed, $\mu \pm \sigma$') 
plt.xticks(x_season, ('SON', 'DJF', 'MAM', 'JJA'))

plt.ylim(0,15)
plt.minorticks_off()

ax.legend()
ax.set_ylabel('Near Surface Wind Speed [m/s]')
ax.set_xlabel('Seasons')
ax.grid(True)
plt.savefig('a-priori_wind_bar1', dpi=450)

# Comparison 4 tested cases:
# case 1 ////////////////////
# ploting graph 
fig, ax = plt.subplots(1)
plt.scatter(x, y_true1, marker='s', color='green', alpha = 0.3, label='True') 
plt.errorbar(x, y_a, 
             yerr = y_a_error, 
             fmt ='o', capsize=8 , color='orange', label='A-priori') 
plt.errorbar(x, y_op1, 
             yerr = y_op_error1, 
             fmt ='o', capsize=5, fillstyle='left', color='blue', alpha = 0.5, label='Optimal') 
plt.xticks(x, ('loc1, SON', 'loc2, DJF', 'loc3, DJF', 'loc4, MAM', 'loc5, JJA'))

plt.ylim(0,21)
#plt.xlim(0,7)
plt.minorticks_off()

ax.legend()
ax.set_ylabel('Near Surface Wind Speed [m/s]')
ax.set_xlabel('Locations for test')
ax.grid(True)
plt.savefig('all_wind_bar1', dpi=450)

# case 2 //////////////////////////
# ploting graph 
fig, ax = plt.subplots(1)
plt.scatter(x, y_true1, marker='s', color='green', alpha = 0.3, label='True') 
plt.errorbar(x, y_a, 
             yerr = y_a_error, 
             fmt ='o', capsize=8 , color='orange', label='A-priori') 
plt.errorbar(x, y_op2, 
             yerr = y_op_error2, 
             fmt ='o', capsize=5, fillstyle='left', color='blue', alpha = 0.5, label='Optimal') 
plt.xticks(x, ('loc1, SON', 'loc2, DJF', 'loc3, DJF', 'loc4, MAM', 'loc5, JJA'))

plt.ylim(0,21)
#plt.xlim(0,7)
plt.minorticks_off()

ax.legend()
ax.set_ylabel('Near Surface Wind Speed [m/s]')
ax.set_xlabel('Locations for test')
ax.grid(True)
plt.savefig('all_wind_bar2', dpi=450)

# case 3 ///////////////////////
# ploting graph 
fig, ax = plt.subplots(1)
plt.scatter(x, y_true1, marker='s', color='green', alpha = 0.3, label='True') 
plt.errorbar(x, y_a, 
             yerr = y_a_error, 
             fmt ='o', capsize=8 , color='orange', label='A-priori') 
plt.errorbar(x, y_op3, 
             yerr = y_op_error3, 
             fmt ='o', capsize=5, fillstyle='left', color='blue', alpha = 0.5, label='Optimal') 
plt.xticks(x, ('loc1, SON', 'loc2, DJF', 'loc3, DJF', 'loc4, MAM', 'loc5, JJA'))

plt.ylim(0,21)
#plt.xlim(0,7)
plt.minorticks_off()

ax.legend()
ax.set_ylabel('Near Surface Wind Speed [m/s]')
ax.set_xlabel('Locations for test')
ax.grid(True)
plt.savefig('all_wind_bar3', dpi=450)

# case 4 ///////////////////////////
# ploting graph 
fig, ax = plt.subplots(1)
plt.scatter(x, y_true1, marker='s', color='green', alpha = 0.3, label='True') 
plt.errorbar(x, y_a, 
             yerr = y_a_error, 
             fmt ='o', capsize=8 , color='orange', label='A-priori') 
plt.errorbar(x, y_op4, 
             yerr = y_op_error4, 
             fmt ='o', capsize=5, fillstyle='left', color='blue', alpha = 0.5, label='Optimal') 
plt.xticks(x, ('loc1, SON', 'loc2, DJF', 'loc3, DJF', 'loc4, MAM', 'loc5, JJA'))

plt.ylim(0,21)
#plt.xlim(0,7)
plt.minorticks_off()

ax.legend()
ax.set_ylabel('Near Surface Wind Speed [m/s]')
ax.set_xlabel('Locations for test')
ax.grid(True)
plt.savefig('all_wind_bar4', dpi=450)

# all 4 cases: //////////////////////
# ploting graph 
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)
#plt.scatter(x, y_true1, marker='s', color='green', alpha = 0.3, label='True') 

ax1.plot(x, np.absolute(np.array(y_true1)-np.array(y_op1)), marker ='o', 
            color='blue', alpha = 0.5, label='All ch') 
ax2.plot(x, y_op_error1, marker ='s', 
            color='blue', alpha = 0.5, label='All ch') 
ax1.plot(x, np.absolute(np.array(y_true1)-np.array(y_op2)), marker ='o', 
            color='red', alpha = 0.5, label='All ch - h_var') 
ax2.plot(x, y_op_error2, marker ='s', 
            color='red', alpha = 0.5, label='All ch - h_var') 
ax1.plot(x, np.absolute(np.array(y_true1)-np.array(y_op3)), marker ='o', 
            color='green', alpha = 0.5, label='Window ch') 
ax2.plot(x, y_op_error3, marker ='s', 
            color='green', alpha = 0.5, label='Window ch') 
ax1.plot(x, np.absolute(np.array(y_true1)-np.array(y_op4)), marker ='o', 
            color='orange', alpha = 0.5, label='Absorp ch') 
ax2.plot(x, y_op_error4, marker ='s', 
            color='orange', alpha = 0.5, label='Absorp ch') 

ax1.set_xticks(x)
ax2.set_xticks(x)
ax1.set_xticklabels(['loc1, SON', 'loc2, DJF', 'loc3, DJF', 'loc4, MAM', 'loc5, JJA'], rotation=30)
ax2.set_xticklabels(['loc1, SON', 'loc2, DJF', 'loc3, DJF', 'loc4, MAM', 'loc5, JJA'], rotation=30)

ax1.grid(True)
ax2.grid(True)


ax1.set_title('abs(True-Optimal)')
ax2.set_title('Uncertainty (1 $\sigma$)')

ax2.legend()

ax1.set_ylabel('Error in Wind Speed [m/s]')
#ax.set_xlabel('Locations for test')
#ax.grid(True)
plt.savefig('all_wind_errors', dpi=450)